In [4]:
from pattern.de import parse, split
import re
import pandas as pd
import nltk

In [13]:
PATH_TO_DATASET = "../../data/protocols/all_parsed.csv"
PATH_TO_ADJECTIVES_DATASET = "adjektive/nearly_all.csv"

In [7]:
df = pd.read_csv(PATH_TO_DATASET)

In [10]:
def contains_superlative(sentence):
    res = re.search('am [a-zäöüß]+(s|ß)ten',sentence)
    
    return bool(res)

def contains_comparative(sentence, comparative_words = None):
    
    res_list = []
    
    # First case: so ... wie
    res_case_1 = re.search('so [a-zäöüß ]* wie',sentence)
    res_list.append(bool(res_case_1))
    
    # Second case: nicht so ... wie
    res_case_2 = re.search('nicht so [a-zäöüß ]* wie',sentence)
    res_list.append(bool(res_case_2))
    
    # Third case: immer ...
    res_case_3 = re.search('immer [a-zäöüß]+er',sentence)
    res_list.append(bool(res_case_3))
    
    # Fourth case: als
    res_case_4 = re.search('[a-zäöüß]+er als [a-zäöüß]+',sentence)
    res_list.append(bool(res_case_4))
    
    # Fifth case: je ... desto or je ... umso
    res_case_5 = re.search('(Je|je) [a-zäöüß ,]+(desto|umso)',sentence)
    res_list.append(bool(res_case_5))
    
    # Sixth case: text contains any word that is specified in the set of comparative words
    # optional, as the set of comparative words is an optional parameter
    if comparative_words is not None:
        words = nltk.tokenize.word_tokenize(sentence,language='german')
        
        for word in words:
            if word in comparative_words:
                res_list.append(True)
                break
    
    
    return True in res_list

In [16]:
df_adjectives = pd.read_csv(PATH_TO_ADJECTIVES_DATASET)

In [33]:
set_comparative = set(df_adjectives["komparativ"].to_list())

In [34]:
words_to_remove = []

for word in set_comparative:
    if len(word) < 3:
        words_to_remove.append(word)

In [36]:
for word in words_to_remove:
    set_comparative.remove(word)

In [76]:
df["speech"]

0        Bevor ich die Sitzung schließe, meine sehr gee...
1        Wir haben eine neue Regierung angelobt. Das sc...
2        Ich darf mich ganz, ganz herzlich dafür bedank...
3        Wir konnten ab Mai unsere Arbeit wieder sukzes...
4        Ich denke da an die Veranstaltungen, die Arbei...
                               ...                        
63904    Meine Damen und Herren, Sie sehen also: Opposi...
63905    Abgeordneter Michel Reimon, MBA (Grüne) : Herr...
63906    Wie alle Vorredner gesagt haben: Alle Daten, d...
63907    Wir haben das als Oppositionspartei auch gesch...
63908    Wir stellen uns schützend vor die Polizei und ...
Name: speech, Length: 63909, dtype: object

In [25]:
df["contains_comparative"] = df["speech"].apply(lambda row: contains_comparative(row))

In [37]:
df["contains_comparative_with_set"] = df["speech"].apply(lambda row: contains_comparative(row,set_comparative))

In [54]:
df["contains_superlative"] = df["speech"].apply(lambda row: contains_superlative(row))

In [56]:
df_comp_sup = df[df["contains_superlative"] & df["contains_comparative_with_set"]]

In [65]:
df_comp_sup.iloc[52]

speaker                          Bundesministerin für Digitalisierung und Wirts...
speech                           Wenn wir darauf schauen, was wir gemeinsam err...
file                                                           NRSITZ_00085_PARSED
contains_comparative_with_set                                                 True
contains_comparative                                                         False
comp_sup                                                                      True
contains_superlative                                                          True
Name: 14348, dtype: object

In [89]:
df[df["contains_comparative"]]["speech"]

6        Ich darf mich bei den Mitgliedern der Präsidia...
103      Das ist eben das, wie die ÖVP das am laufenden...
106      Da sieht man, wie die schwarzen Netzwerke so a...
111      Schiebt man diesen Fall jetzt zur Seite, Herr ...
115      Herr Bundeskanzler, darum geht es uns, darum g...
                               ...                        
63786    Wenn bei Kindern und Jugendlichen mit einer un...
63803    Sehr geehrter Herr Präsident! Sehr geschätzte ...
63830    Sehr geehrter Herr Präsident! Sehr geehrter He...
63866    Ich freue mich, dass die ÖVP bereits Sympathie...
63876    Leider ist das nur eine Ausnahme, denn auch im...
Name: speech, Length: 3130, dtype: object